In [146]:
import os
import numpy as np
import sys
import socket
import pickle
import math
import os
import json
from tqdm import tqdm
import torch
import k3d

from pytorch3d.io import load_obj,save_ply
from pytorch3d.ops import sample_points_from_meshes,knn_points, sample_farthest_points
from pytorch3d.structures import Pointclouds,Meshes

In [147]:
pix_path = '/scratch/fml35/datasets/pix3d_new/'
with open("/data/cornucopia/fml35/experiments/exp_024_debug/models/model_list.json",'r') as f:
        model_list = json.load(f)["models"]

# for j in tqdm(range(0,1)):#len(model_list))):
j = 40
path = pix_path + model_list[j]["model"]

In [148]:
verts_torch,faces_torch,_ = load_obj(path,load_textures=False)

verts = verts_torch.numpy()
faces = faces_torch[0].numpy()

In [149]:
n_points = 50000
n_nn = 5
angle_threshold_degree = 35


mesh = Meshes(verts=[verts_torch],faces=[faces_torch[0]])
sample_points,sample_normals = sample_points_from_meshes(mesh,n_points,return_normals=True)
sample_points = sample_points.squeeze()
sample_normals = sample_normals.squeeze()

In [150]:

# find distance between two pointclouds
# For each predicted point, find its neareast-neighbor GT point
knn_pred = knn_points(sample_points.unsqueeze(0),sample_points.unsqueeze(0), K=n_nn)

sample_normals_unsq = sample_normals.unsqueeze(1)
sample_normals_tiled = sample_normals_unsq.tile(1,n_nn,1)

nn_normals = torch.zeros(n_points,n_nn,3)
for i in tqdm(range(n_points)):
    n = sample_normals[knn_pred.idx[0,i]]
    nn_normals[i] = n
    
normal_dot = torch.sum(sample_normals_tiled*nn_normals,dim=2)
angles = torch.arccos(normal_dot) * 180 / np.pi
angles = torch.min(angles,torch.abs(180. - angles))
# TODO: think if better 180 or 360
# angles = torch.min(angles,torch.abs(360. - angles))
max_angles,_ = torch.max(angles,dim=1)
labels = (max_angles > angle_threshold_degree)
print(labels.shape)



100%|██████████| 50000/50000 [00:00<00:00, 61042.15it/s]

torch.Size([50000])


In [151]:

plot = k3d.plot()

# plot += k3d.points(sample_points.numpy()[~labels],point_size=0.01,color=(255))
plot += k3d.points(sample_points[labels],point_size=0.005,color=(0 + 256*255))
plot.display()

Output()

In [152]:

plot = k3d.plot()

points,_ = sample_farthest_points(sample_points[labels].unsqueeze(0),K = 1000)
# plot += k3d.points(sample_points.numpy()[~labels],point_size=0.01,color=(255))
plot += k3d.points(points.squeeze(),point_size=0.005,color=(0 + 256*255))
plot.display()

Output()

In [153]:
def dist_point_line(lp,p):
    # lp: N_lines x 2 (x1,y1,x2,y2) and p: N_points x 2
    N_l = lp.shape[0]
    N_p = p.shape[0]

    x1 = lp[:,0].repeat(N_p)
    y1 = lp[:,1].repeat(N_p)
    z1 = lp[:,2].repeat(N_p)
    x2 = lp[:,3].repeat(N_p)
    y2 = lp[:,4].repeat(N_p)
    z2 = lp[:,5].repeat(N_p)
    x0 = p[:,0].repeat_interleave(N_l)
    y0 = p[:,1].repeat_interleave(N_l)
    z0 = p[:,2].repeat_interleave(N_l)


    px = x2-x1
    py = y2-y1
    pz = z2-z1

    norm = px*px + py*py + pz*pz

    # print('norm',norm)

    u =  ((x0 - x1) * px + (y0 - y1) * py + (z0 - z1) * pz) / norm

    u = torch.clip(u,min=0,max=1)

    # print('u',u)

    x = x1 + u * px
    y = y1 + u * py
    z = z1 + u * pz

    dx = x - x0
    dy = y - y0
    dz = z - z0



    dist_non_flattened = (dx*dx + dy*dy + dz*dz)#**.5

    # print('dist_non_flatten',dist_non_flattened)
    # dist = torch.abs(dx) + torch.abs(dy) + torch.abs(dz)

    # dist = torch.abs((x2 - x1) * (y1 - y0) - (x1 - x0) * (y2 - y1)) / torch.sqrt((x2-x1)**2 + (y2 -y1)**2)
    dist_non_flattened = dist_non_flattened.reshape(N_p,N_l)
    dist,_ = torch.min(dist_non_flattened,dim=1)

    return dist,dist_non_flattened

In [154]:
lp = torch.Tensor([[0,0,0,0,1,0],[1,0,0,1,1,0]])
p = torch.Tensor([[0,0,0],[0.7,0.5,0],[0.5,1,0],[0.5,2,0]])
print(dist_point_line(lp,p))

(tensor([0.0000, 0.0900, 0.2500, 1.2500]), tensor([[0.0000, 1.0000],
        [0.4900, 0.0900],
        [0.2500, 0.2500],
        [1.2500, 1.2500]]))


In [155]:
def create_plot(points_optimise,line_points,mask,distances):
    plot = k3d.plot()

    plot += k3d.points(points_optimise,point_size=0.03,color=(0 + 256*255))
    plot += k3d.points(points_optimise[distances > 0.00125],point_size=0.03,color=(255))
    for i in range(n_lines):
        if mask is None or mask[i] == True: 
            plot += k3d.line([line_points[i,:3].numpy(), line_points[i,3:6].numpy()], shader='mesh', width=.003, color=0xff0000)
    plot.display()

In [156]:
def sample_points_from_lines(lines,points_per_line):
    n_lines = lines.shape[0]
    lines = torch.repeat_interleave(lines,points_per_line,dim=0)
    interval = torch.linspace(0,1,points_per_line).repeat(n_lines)
    interval = interval.unsqueeze(1).repeat(1,3)
    points = lines[:,:3] + (lines[:,3:6] - lines[:,:3]) * interval
    return points

In [157]:

def find_line(points,dist_threshold_squared,min_n_support):
    print(points.shape)
    N = points.shape[0]

    #
    start_id = torch.randint(low=0,high=N,size=(1,))
    lp1 = points[start_id].tile(N,1)
    lp = torch.cat([lp1,points],dim=1)
    distances,non_flat = dist_point_line(lp,points)
    thresholded = non_flat < dist_threshold_squared
    n_support = torch.sum(thresholded,dim=0)
    best_index = torch.argmax(n_support)

    # now repeat but
    lp1 = points[best_index].tile(N,1)
    lp = torch.cat([lp1,points],dim=1)
    distances,non_flat = dist_point_line(lp,points)
    thresholded_2 = non_flat < dist_threshold_squared
    n_support = torch.sum(thresholded_2,dim=0)
    sorted_support,indices = torch.sort(n_support,descending=True)

    # print('sorted',sorted[:10])
    # print('indices',indices[:10])

    found = False
    for i in range(N):
        test_index = indices[i]
        # print(test_index)
        # IDEA: use double distance or triple to allow for better lines, ie. 2x threshold
        if thresholded_2[start_id,test_index] == True and sorted_support[i] >= min_n_support:
            print('N support',sorted_support[i])
            found = True
            break

    if found == False:
        return None,points
        
    else:
        # print('best_index',best_index)
        # print('test_index',test_index)
        line = torch.cat([points[best_index],points[test_index]])
        # print('line',line)
        remaining_points = points[thresholded[:,best_index].nonzero()]
        remaining_points_2 = points[(~thresholded_2[:,test_index]).nonzero()]

        return line,remaining_points_2.squeeze()



    plot = k3d.plot()

    # plot += k3d.points(points,point_size=0.01,color=(0 + 256*255))
    # start blue
    plot += k3d.points(points[start_id],point_size=0.04,color=(255))
    plot += k3d.points(line[:3],point_size=0.03,color=(0 + 256*256*255))
    plot += k3d.points(line[3:6] + torch.Tensor([0.03,0,0]),point_size=0.03,color=(255 + 256*256*255))
    # plot += k3d.points(points[best_index],point_size=0.03,color=(0 + 256*256*255))
    # plot += k3d.points(points[test_index],point_size=0.03,color=(255 + 256*256*255))
    # plot += k3d.points(points,point_size=0.01,color=(256*255 + 256*256*255))
    # plot += k3d.points(remaining_points,point_size=0.01,color=(256*256*255))
    plot += k3d.points(remaining_points_2,point_size=0.01,color=(255+ 256*256*255))
    plot.display()


dist_threshold_squared = 0.015**2
min_n_support = 10
n_lines = 50
torch.manual_seed(5)

# remaining_points = torch.from_numpy(sample_points.numpy()[labels])
remaining_points = points.squeeze()


lines = torch.zeros((n_lines,6))
counter = 0
for i in range(n_lines):
    line,remaining_points = find_line(remaining_points,dist_threshold_squared,min_n_support)
    if line != None:
        lines[counter] = line
        counter += 1


plot = k3d.plot()
plot += k3d.points(remaining_points,point_size=0.01,color=(255+ 256*256*255))
plot.display()

plot = k3d.plot()
plot += k3d.points(sample_points_from_lines(lines,points_per_line=100),point_size=0.01,color=(255+ 256*256*255))
plot.display()



torch.Size([1000, 3])
N support tensor(117)
tensor([ 0.1723,  0.0211,  0.1357, -0.1918,  0.0125, -0.1530])
torch.Size([883, 3])
N support tensor(99)
tensor([ 0.2007,  0.0017, -0.1466,  0.2433, -0.3843, -0.1921])
torch.Size([784, 3])
N support tensor(65)
tensor([ 0.2504, -0.3682,  0.1831,  0.1513,  0.1267,  0.1587])
torch.Size([719, 3])
N support tensor(120)
tensor([-0.1980,  0.0074,  0.1515,  0.1647,  0.0233, -0.1277])
torch.Size([599, 3])
N support tensor(108)
tensor([-0.2462, -0.3879,  0.1953, -0.1975,  0.1005,  0.1574])
torch.Size([491, 3])
N support tensor(52)
tensor([ 0.2155, -0.1434,  0.1576,  0.1892,  0.3680,  0.1894])
torch.Size([439, 3])
N support tensor(25)
tensor([ 0.1895,  0.0131, -0.1385, -0.1975,  0.0550,  0.1424])
torch.Size([414, 3])
N support tensor(29)
tensor([ 0.2040,  0.0556,  0.1463,  0.1861,  0.0399, -0.1256])
torch.Size([385, 3])
N support tensor(34)
tensor([ 0.1789,  0.3762,  0.1981, -0.1753,  0.3752,  0.1908])
torch.Size([351, 3])
N support tensor(26)
tensor([-

Output()

Output()

In [158]:
# print(start.shape)
line_points = start.clone()
line_points.requires_grad = True
# points_optimise = torch.Tensor([[0,0,0],[1,1,1],[0,0.2,0.4],[0,0.23,0.4]])

# optimizer = torch.optim.SGD([line_points], lr=1e2,momentum=0.0,dampening=0, weight_decay=0, nesterov=False)
optimizer = torch.optim.Adam([line_points], lr=1e-1)
print(points_optimise.shape)
for step in tqdm(range(n_steps)):
        
    optimizer.zero_grad()

    distances,non_flat = dist_point_line(line_points,points_optimise)
    # print('distances.shape',distances[:5])
    # print('non_flat.shape',non_flat[:5])
    # print('distances.tile(1,n_lines)',distances.tile(1,n_lines).shape)
    mask = distances.unsqueeze(1).tile(1,n_lines) == non_flat
    mask_all_points = mask.any(dim=0)
    # print(mask)
    # print(mask.shape)
    # print(mask_all_points)

    if (step ) % 20 == 0:
        # print('line_points',line_points)
        # print('points_optimise',points_optimise[:3])
        # print('non flat',non_flat[:6])
        print('dists ',distances[:])
        # print('loss',loss)
        create_plot(points_optimise,line_points.detach(),mask_all_points,distances)
        print(torch.sum(mask_all_points))
        # print(mask)
        # print(mask_all_points.shape)
        # print(mask_all_points)
        pass
    

    loss = torch.mean(distances)
    loss.backward()
    optimizer.step()

    # line_points = line_points.detach()

    # for i in range(n_lines):
    #     if not mask_all_points[i]:
    #         line_points[i] = torch.rand((6)) - 0.5
        
    # line_points.requires_grad = True

# create_plot(points_optimise,line_points.detach(),None)
create_plot(points_optimise,line_points.detach(),mask_all_points)



torch.Size([5310, 3])


  0%|          | 0/1000 [00:00<?, ?it/s]


RuntimeError: The size of tensor a (50) must match the size of tensor b (10) at non-singleton dimension 1